In [1]:
import sys
sys.path.append("D:/gnn-cloud-manufacturing/")

import torch
from torch.optim import Adam
from tqdm.auto import trange
import matplotlib.pyplot as plt
from tqdm.auto import trange
from cloudmanufacturing.data import read_fatahi_dataset
from cloudmanufacturing.validation import objvalue, construct_delta, Dataset, validate
from cloudmanufacturing.graph import os_type
from cloudmanufacturing.graphconv import GNN
import dgl
from dgl.dataloading import GraphDataLoader
import pickle
import torch.nn.functional as F
import logging
import torch
from torch.optim import Adam


c:\Users\ASUS Vivobook\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('../data/train_dataset/train_data_sheet_names.pickle' , 'rb') as s:
    sheet_names = pickle.load(s)
dataset = read_fatahi_dataset('../data/train_dataset/train_data_OPTIMAL.xlsx', sheet_names)

100%|██████████| 54/54 [00:24<00:00,  2.23it/s]


In [3]:
with open('../data/train_dataset/train_data_solve.pickle' , 'rb') as f:
    # Загружаем dgl графы в список
    DGList = pickle.load(f)

In [4]:
DGList.pop(26)
dataset.pop(26)

{'name': '3,24,19-11',
 'n_tasks': 3,
 'n_operations': 24,
 'n_cities': 19,
 'n_services': 1,
 'operation': array([[0., 1., 1.],
        [0., 1., 1.],
        [1., 1., 1.],
        [1., 1., 0.],
        [0., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 0.],
        [0., 1., 0.],
        [1., 1., 1.],
        [0., 1., 0.],
        [1., 1., 1.],
        [0., 1., 1.],
        [1., 1., 0.],
        [1., 0., 1.],
        [1., 1., 1.],
        [0., 1., 0.],
        [1., 1., 0.],
        [1., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 1.],
        [1., 1., 0.],
        [1., 1., 0.]]),
 'dist': array([[   0., 1749., 1493.,  498.,  498., 2057., 1820., 1859., 1205.,
          665.,  791., 1959.,  903., 2089.,  883., 1633.,  598., 1655.,
         1298.],
        [1749.,    0., 1214., 1649., 1663., 1129.,  336.,  717., 1472.,
         1125., 1732., 1010., 1855.,  961., 1216., 1044., 1744.,  440.,
         1650.],
        [1493., 1214.,    0., 175

In [5]:
train_dataset = Dataset(DGList[:45], dataset[:45])
val_dataset = Dataset(DGList[45:], dataset[45:])

In [6]:
logger = logging.getLogger("../data/GNN_run")
logger.setLevel(logging.INFO)

fh = logging.FileHandler("../data/GNN_run.log")

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)

logger.addHandler(fh)

In [7]:
n_epochs = 5000
# Списки значений параметров, которые вы хотите протестировать
lr_values = [0.001, 0.01, 0.1]
n_layers_values = [1, 2, 3]
batch_size_values = [4, 8]

results_file = open("../data/results_objvalues.txt", "a")

logger.info('Start training')

try:

    for lr in lr_values:
        for n_layers in n_layers_values:
            for batch_size in batch_size_values:
                # Создание модели с новыми параметрами
                model = GNN(ins_dim=1, ino_dim=20, out_dim=16, n_layers=n_layers)
                optim = Adam(model.parameters(), lr=lr)
                
                train_loader = GraphDataLoader(
                    train_dataset,
                    batch_size=batch_size,
                    drop_last=False,
                    shuffle=True
                )

                val_loader = GraphDataLoader(
                    val_dataset,
                    batch_size=batch_size,
                    drop_last=False,
                    shuffle=False
                )

                # Здесь должен быть код для загрузки данных и создания DataLoader'ов

                # Запуск обучения
                for epoch in trange(n_epochs):
                    train_objvalue = []
                    for batch, idx in train_loader:
                        logits = model(batch)
                        target = batch.edata['target'][os_type]
                        loss = F.binary_cross_entropy_with_logits(logits, target)
                        batch_loss = loss.item()
                        optim.zero_grad()
                        loss.backward()
                        optim.step()
                        logger.info(f"{epoch} epoch batch loss is {batch_loss}")

                        # Validation

                        # Save results to file

                        
                        for i, graph in enumerate(dgl.unbatch(batch)):
                            problem = dataset[idx[i]]
                            with graph.local_scope():
                                pred_gamma = model.predict(graph, problem)
                            train_objvalue.append(
                                objvalue(problem, pred_gamma, construct_delta(problem, pred_gamma))
                            )

                    

                    train_loss = sum(train_objvalue)/len(train_objvalue)
                    val_loss = validate(model, val_loader, dataset)

                    if epoch%10==0:
                        #print('Epoch: ', epoch, 'Train: ', train_loss, 'Val: ', val_loss)  
                        #print(f"batch_size: {batch_size}, n_layers: {n_layers}, lr: {lr}, epoch: {epoch}, train_loss: {train_loss}, val_loss: {val_loss}\n")
                        results_file.write(f"batch_size: {batch_size}, n_layers: {n_layers}, lr: {lr}, epoch: {epoch}, train_loss: {train_loss}, val_loss: {val_loss}\n")
                
                Path = "../data/model_" + f"batch_size:{batch_size},n_layers:{n_layers},lr:{lr},epoch:{epoch},train_loss:{train_loss},val_loss:{val_loss}.pth"
                torch.save(model.state_dict(), Path)
    results_file.close()

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    results_file.close()


  0%|          | 0/5000 [00:00<?, ?it/s]C:\Users\ASUS Vivobook\AppData\Roaming\Python\Python310\site-packages\dgl\backend\pytorch\tensor.py:352: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), "Cannot convert view " \
  0%|          | 1/5000 [00:02<3:09:19,  2.27s/it]

Epoch:  0 Train:  3743.0914610979335 Val:  7290.065633744426
batch_size: 4, n_layers: 1, lr: 0.001, epoch: 0, train_loss: 3743.0914610979335, val_loss: 7290.065633744426

../data/model_batch_size:4,n_layers:1,lr:0.001,epoch:0,train_loss:3743.0914610979335,val_loss:7290.065633744426.pth


  0%|          | 11/5000 [00:24<2:56:34,  2.12s/it]

Epoch:  10 Train:  3689.277929382898 Val:  7286.879655723892
batch_size: 4, n_layers: 1, lr: 0.001, epoch: 10, train_loss: 3689.277929382898, val_loss: 7286.879655723892

../data/model_batch_size:4,n_layers:1,lr:0.001,epoch:10,train_loss:3689.277929382898,val_loss:7286.879655723892.pth


  0%|          | 21/5000 [00:46<3:21:15,  2.43s/it]

Epoch:  20 Train:  3441.9447285519823 Val:  6557.321866067185
batch_size: 4, n_layers: 1, lr: 0.001, epoch: 20, train_loss: 3441.9447285519823, val_loss: 6557.321866067185

../data/model_batch_size:4,n_layers:1,lr:0.001,epoch:20,train_loss:3441.9447285519823,val_loss:6557.321866067185.pth


  0%|          | 25/5000 [00:57<3:10:50,  2.30s/it]


KeyboardInterrupt: 